In [ ]:
import torch
import torch.nn as nn

# Define the basic building blocks: Residual Block
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = None

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self.make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

        # Add two fully connected layers for 1x512 outputs
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 512)

    def make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        
        # Pass through two fully connected layers for 1x512 outputs
        out1 = self.fc1(out)
        out2 = self.fc2(out)
        
        # Concatenate the two 1x512 outputs to get a 1x1024 array
        concatenated_output = torch.cat((out1, out2), dim=1)
        
        # Apply the final classification layer
        output = self.fc(concatenated_output)
        return output

# Create a ResNet model
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# Instantiate the model
model = ResNet18()

Files already downloaded and verified
Epoch 1/1, Batch 0/782, Loss: 2.4357717037200928
Epoch 1/1, Batch 100/782, Loss: 1.9979636669158936
Epoch 1/1, Batch 200/782, Loss: 2.032679557800293
Epoch 1/1, Batch 300/782, Loss: 1.7990580797195435
Epoch 1/1, Batch 400/782, Loss: 1.906455397605896
Epoch 1/1, Batch 500/782, Loss: 1.5445990562438965
Epoch 1/1, Batch 600/782, Loss: 1.5406376123428345
Epoch 1/1, Batch 700/782, Loss: 1.5161807537078857


In [ ]:
test_loader= model.forward(Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900))
# Evaluation loop
def evaluate(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            # Calculate the number of correct predictions
            _, predicted = outputs.max(1)
            correct_predictions += predicted.eq(targets).sum().item()

            total_samples += targets.size(0)

    # Calculate accuracy and average loss
    accuracy = (correct_predictions / total_samples) * 100.0
    average_loss = total_loss / len(test_loader)

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Average Test Loss: {average_loss:.4f}")

# Call the evaluation function
evaluate(model, test_loader, criterion)


Files already downloaded and verified
Test Accuracy: 44.64%
Average Test Loss: 1.5221


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from IPython.display import Image, clear_output

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Flatten, Embedding, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Bidirectional(LSTM(512, activation="tanh", return_sequences=True)),
    Bidirectional(LSTM(256, activation="tanh", return_sequences=True)),
    Bidirectional(LSTM(128, activation="tanh")),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(2, activation="softmax")
])

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
x_train= Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)
# y_train is 0 if P2 wins and 1 if P1 wins
hist = model.fit(x_train, 1, epochs=1000)